In [7]:
import sqlite3
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

dat = sqlite3.connect('FPA_FOD_20170508.sqlite')
query = dat.execute("SELECT * From Fires")
cols = [column[0] for column in query.description]
df = pd.DataFrame(data = query.fetchall(), columns = cols)


In [8]:
df.size

73338135

In [9]:
data_num = df.select_dtypes(include=[np.number])
data_cat = df.select_dtypes(exclude=[np.number])

num_attribs = list(data_num)
cat_attribs = list(data_cat)
cat_attribs.remove("FIRE_SIZE_CLASS")

print("num", num_attribs)
print("cat", cat_attribs)

num ['OBJECTID', 'FOD_ID', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'STAT_CAUSE_CODE', 'CONT_DATE', 'CONT_DOY', 'FIRE_SIZE', 'LATITUDE', 'LONGITUDE', 'OWNER_CODE']
cat ['FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'DISCOVERY_TIME', 'STAT_CAUSE_DESCR', 'CONT_TIME', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME', 'Shape']


In [10]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(df['STATE'].values.reshape(-1,1))
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['STATE']))

df = df.drop(columns=cat_attribs)
df = df.drop(columns=['OBJECTID', 'FOD_ID', 'FIRE_YEAR', 'DISCOVERY_DATE', 'CONT_DATE', 'CONT_DOY'])

df = pd.concat([df, one_hot_df], axis=1)

# corr_matrix = df.corr(method='spearman')
# sorted_corr_matrix = corr_matrix['FIRE_SIZE'].sort_values(ascending=False)
# print(sorted_corr_matrix)
# top_attributes = sorted_corr_matrix.index.tolist()

# for attribute in top_attributes:
#     df_corr.plot(kind="scatter", x=attribute, y="FIRE_SIZE", alpha=0.1)

# plt.show()


In [11]:
#drops = ['FIRE_SIZE', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'DISCOVERY_TIME', 'STAT_CAUSE_DESCR', 'CONT_TIME', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME', 'Shape', 'OBJECTID', 'FOD_ID', 'FIRE_YEAR', 'DISCOVERY_DATE', 'CONT_DATE', 'CONT_DOY']
drops = ['FIRE_SIZE']
testing_df = df.drop(columns=drops, axis=1)
testing_df.dropna()
testing_df.size
testing_df.select_dtypes(include=[np.number])

,DISCOVERY_DOY,STAT_CAUSE_CODE,LATITUDE,LONGITUDE,OWNER_CODE,STATE_AK,STATE_AL,STATE_AR,STATE_AZ,STATE_CA,...,STATE_SD,STATE_TN,STATE_TX,STATE_UT,STATE_VA,STATE_VT,STATE_WA,STATE_WI,STATE_WV,STATE_WY
0,33,9.0,40.036944,-121.005833,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,133,1.0,38.933056,-120.404444,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,152,5.0,38.984167,-120.735556,13.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,180,1.0,38.559167,-119.913333,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,180,1.0,38.559167,-119.933056,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,269,13.0,40.481637,-122.389375,13.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1880461,278,9.0,37.617619,-120.938570,12.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1880462,122,13.0,37.617619,-120.938570,12.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1880463,287,13.0,37.672235,-120.898356,12.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
from sklearn.model_selection import train_test_split

X = testing_df.drop(columns=['FIRE_SIZE_CLASS']).values
y = testing_df['FIRE_SIZE_CLASS'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05, random_state=42)

In [13]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Logistic Regression Accuracy:", accuracy)

Logistic Regression Accuracy: 0.5494448225984855


In [14]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Decision Tree Accuracy:", accuracy)

Decision Tree Accuracy: 0.5495618140049349


In [15]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=100, weights='distance') 

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("KNN Accuracy:", accuracy)

KNN Accuracy: 0.6269569471624267


In [16]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, random_state=1, min_samples_split=4, max_depth=50)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print("Random Forest Accuracy:", accuracy)

Random Forest Accuracy: 0.6366885050625373


In [17]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=42, alpha = 0.000001)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Neural Network Accuracy:", accuracy)

Neural Network Accuracy: 0.6167574236365183
